In [1]:
import sys
import os
import importlib
import time
import random
import cython
import pickle
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer #计算tfidf
from sklearn.feature_extraction.text import CountVectorizer  #计算df
from sklearn.feature_extraction.text import TfidfVectorizer  #“一步到位”
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib # to save
from sklearn import metrics

/home/lihaoyuan/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [10]:
id_num = []
word_article = []
words_article = []
label = []
X_train = []
y_train = []
X_test = []
y_test = []

load_start = time.time()
importlib.reload(sys)
with open('data/train_set.csv','r',encoding = 'utf-8') as f:
    lines = f.readlines()[1:]
#     random.shuffle(lines) #注意用法
    for line in lines[1:80000]:
        line = line.split(',')
        X_train.append(line[2])
        y_train.append(line[3])
    for line in lines[80000:]: #划分出测试集
        line = line.split(',')
        X_test.append(line[2])
        y_test.append(line[3])
print("Time consumption on loading:",time.time() - load_start)

Time consumption on loading: 2.557532787322998


### 构造 corpus-bert model

In [85]:
with open('data/train_set.csv','r',encoding = 'utf-8') as f, open('data/test_set.csv','r',encoding = 'utf-8') as f2:
    train = f.readlines()[1:]
    test = f2.readlines()[1:]
corpus = []
for s in train:
    corpus.append(s.split(',')[2])#只留下word-level，不要article-level
for s in test:
    corpus.append(s.split(',')[2].strip()) #test_set有'\n'
print(len(train), len(test), len(corpus))

#保存pkl，但是和pytorch-bert用w不符合，我先按照他的来
# pickle.dump(corpus, open('data/corpus.pkl', 'w', encoding='utf-8'))
# corpus = pickle.load(open('data/corpus.pkl', 'r'))
# with open('data/corpus.pkl', 'r', encoding='utf-8') as f:
#     a = f.readlines()


#保存txt
with open('data/corpus.txt', 'w') as f:
    for s in corpus:
        half = len(s)//2
        f.write(s[:half]+' \t '+s[half:]+'\n') #要注意\t左右是否有空格

102277 102277 204554


In [91]:
with open('output/bert.model.ep0', 'r') as f:
    a = f.readlines()

In [95]:
a[:5000]

[b'\x80\x02\x8a\n',
 b'l\xfc\x9cF\xf9 j\xa8P\x19.\x80\x02M\xe9\x03.\x80\x02}q\x00(X\x10\x00\x00\x00protocol_versionq\x01M\xe9\x03X\r\x00\x00\x00little_endianq\x02\x88X\n',
 b'\x00\x00\x00type_sizesq\x03}q\x04(X\x05\x00\x00\x00shortq\x05K\x02X\x03\x00\x00\x00intq\x06K\x04X\x04\x00\x00\x00longq\x07K\x04uu.\x80\x02(X\x06\x00\x00\x00moduleq\x00cbert_pytorch.model.bert\n',
 b'BERT\n',
 b'q\x01XP\x00\x00\x00/home/lihaoyuan/anaconda3/lib/python3.7/site-packages/bert_pytorch/model/bert.pyq\x02X/\x06\x00\x00class BERT(nn.Module):\n',
 b'    """\n',
 b'    BERT model : Bidirectional Encoder Representations from Transformers.\n',
 b'    """\n',
 b'\n',
 b'    def __init__(self, vocab_size, hidden=768, n_layers=12, attn_heads=12, dropout=0.1):\n',
 b'        """\n',
 b'        :param vocab_size: vocab_size of total words\n',
 b'        :param hidden: BERT model hidden size\n',
 b'        :param n_layers: numbers of Transformer blocks(layers)\n',
 b'        :param attn_heads: number of attention he

In [90]:
#平均长度
mi,ma,mean = 99,0,0
for s in a:
    L = len(s)
    mean+=L
    if L<mi:
        mi=L
        print(s,'a') #这样的句子应该去掉
    if L>ma:
        ma=L
print(mi, ma, mean/len(a))

409779 816903 409779  	 816903 608182 1226448
 a

 a
1 282882 3418.113195863521


## 训练

In [2]:
train_start = time.time()
vect = TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, norm='l2', use_idf=1, smooth_idf=1, sublinear_tf=1)
vect.fit(X_train) # 或者直接X_train = vect.fit_transform(X_train)
X_train = vect.transform(X_train)
# X_train = X_train.toarray()
# pca = PCA(n_components = 10000)
# print(X_train.shape)
# X_train = pca.fit_transform(X_train)
# print(X_train.shape)
# ss=StandardScaler(with_mean=False)
# X_train=ss.fit_transform(X_train)
# X_train = preprocessing.normalize(X_train, norm='l2', axis=1, copy=True) #正则化。。没用
# X_train = preprocessing.scale(X_train, with_mean=False) #标准化
print("Time consumption on training1:",time.time() - train_start)

# clf = svm.LinearSVC(C=0.8)
# clf = svm.SVC(kernel='linear', C=1, )
clf = SGDClassifier(loss="modified_huber", penalty="l2", alpha=0.00002, n_jobs=-1)
        # modified_huber; penalty="elasticnet"
# scores = cross_val_score(clf, X_train, y_train , cv=5, scoring='f1')
# print(scores)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# parameters = {#'loss':('log', 'hinge','modified_huber'),
#               'alpha':[0.00004, 0.00001, 0.00002], #l2时2e-5最优，elasticnet 1e-5最优
#               #'alpha':[0.000006, 0.00001], #1e-5最优
#               #'penalty':['elasticnet','l2']
# }
# grid_search = GridSearchCV(clf, parameters, cv=5)
# grid_search.fit(X_train, y_train)
# best_parameters = dict(grid_search.best_estimator_.get_params())#get实例中的最优参数
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name])) #输出参数结果
clf.fit(X_train, y_train)

# scores = cross_val_score(clf, X_train, y_train , cv=5)
# print(scores)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# joblib.dump(clf, 'lineSVM_20000__.m')
print("Time consumption on training1+2:",time.time() - train_start)

test_start = time.time()
# clf = joblib.load('lineSVM_20000__.m')
prediction = clf.predict(X_train)
print(metrics.classification_report(y_train, prediction))
X_test = vect.transform(X_test)
# X_test = preprocessing.scale(X_test, with_mean=False) 
# X_test = X_test.toarray()
# X_test = preprocessing.normalize(X_test, norm='l2', axis=1, copy=True) #正则化
# X_test=ss.fit_transform(X_test)
# X_test = preprocessing.scale(X_test)
# print(X_test.shape)
prediction = clf.predict(X_test) 
print(metrics.classification_report(y_test,prediction))
# print(metrics.confusion_matrix(y_test,prediction))
print("Time consumption on testing:",time.time() - test_start)

Time consumption on loading: 6.083667516708374
Time consumption on training1: 695.3630359172821


/home/lihaoyuan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Time consumption on training1+2: 707.0833249092102
             precision    recall  f1-score   support

         1
       0.98      0.97      0.98      4233
        10
       0.99      0.99      0.99      3871
        11
       0.98      0.99      0.98      2784
        12
       0.99      0.99      0.99      4170
        13
       0.99      0.99      0.99      6180
        14
       0.99      0.99      0.99      5281
        15
       1.00      1.00      1.00      5953
        16
       0.98      0.98      0.98      2522
        17
       0.98      0.99      0.99      2381
        18
       1.00      1.00      1.00      5519
        19
       0.98      0.98      0.98      4306
         2
       0.99      0.99      0.99      2235
         3
       1.00      1.00      1.00      6496
         4
       0.99      0.99      0.99      3005
         5
       0.99      0.98      0.99      1858
         6
       1.00      1.00      1.00      5370
         7
       0.99      0.99      0.99     